<a href="https://colab.research.google.com/github/kevingray92/ATMS-597-SP-2020/blob/patch-1/ATMS-597-SP-2020-Project-2/ATMS_597_Project_2_Gray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
import requests

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/'+endpoint,
        headers={
            'token': 'yicVcIaiwUAgtBveaBtWSaioiQvqRJRh'
        },
        params=payload
    )

In [0]:
# This cell will request locations. We used this to find the locationid for Champaign, IL area.
# !!!No need to run this cell again unless we want to look up a new locationid!!!

response = make_request(
    'locations', 
    {
        'datasetid' : 'GHCND', 
        'locationcategoryid' : 'CITY',
        'datacategoryid' : 'TEMP',
        'sortorder' : 'desc',
        'limit' : 1000 # max allowed
    }
)
response.json()

In [0]:
# This cell will request stations. We used this to find the stationid for Rantoul, IL station.
# !!!No need to run this cell again unless we want to look up a new stationid!!!

response = make_request(
    'stations', 
    {
        'datasetid' : 'GHCND',
        'locationid' : 'CITY:US170004', 
        'datacategoryid' : 'TEMP',
        'limit' : 1000 # max allowed
    }
)
response.json()

In [0]:
# Create lists containing the beginning and end of years we want to loop over.
# Clunky for now, can probably make this smoother using some kind of loop to add one to the year each time


currentlist = [datetime.date(2015, 1, 1), datetime.date(2016, 1, 1), datetime.date(2017, 1, 1), datetime.date(2018, 1, 1), datetime.date(2019, 1, 1)]
endlist = [datetime.date(2015, 12, 31), datetime.date(2016, 12, 31), datetime.date(2017, 12, 31), datetime.date(2018, 12, 31),datetime.date(2019, 12, 31)]

In [103]:
# This cell will request the data

results = [] # get an empty list to fill with data
numloops = np.arange(len(currentlist)) # fill a numper array with the length of the list of years we want

#Start the loop over the years we want
for i in numloops:
    current = currentlist[i] # set current to the beginning of the year in our loop
    end = endlist[i] # set end to the end of the year in our loop
    # update the cell with status information
    display.clear_output(wait=True)
    display.display(f'Gathering data for {str(current)}')
    response = make_request(
        'data', 
        {
            'datasetid' : 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
            'datatypeid' : 'TMAX',
            'stationid' : 'GHCND:USW00014806',
            'startdate' : current,
            'enddate' : end,
            'units' : 'metric',
            'limit' : 1000 # max allowed
        }
    )
    response.json()
    results.extend(response.json()['results']) # put the data in the results list

'Gathering data for 2019-01-01'

In [104]:
len(results) # check the length of the results list to make sure we have the correct number of days

1826

In [105]:
# Put the results in a pandas dataframe
df = pd.DataFrame(results)
df.head()

,date,datatype,station,attributes,value
0,2015-01-01T00:00:00,TMAX,GHCND:USW00014806,",,7,0700",0.6
1,2015-01-02T00:00:00,TMAX,GHCND:USW00014806,",,7,0700",1.7
2,2015-01-03T00:00:00,TMAX,GHCND:USW00014806,",,7,0700",2.2
3,2015-01-04T00:00:00,TMAX,GHCND:USW00014806,",,7,0700",1.1
4,2015-01-05T00:00:00,TMAX,GHCND:USW00014806,",,7,0700",-7.8


In [0]:
###Rylan's code for getting Yearly Average Temperature goes here. ###